In [1]:
import pandas as pd
import numpy as np
import csv
from collections import defaultdict
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data/historical_transactions.csv')

In [3]:
df.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [40]:
clean = df[['card_id', 'merchant_id', 'purchase_date']].sort_values(by = 'purchase_date', ascending=True)

In [41]:
clean['purchase_date'] = pd.to_datetime(clean['purchase_date'])

In [42]:
clean['purchase_date'].max()

Timestamp('2018-02-28 23:59:51')

In [43]:
clean = clean[clean.purchase_date < clean['purchase_date'].min() + pd.to_timedelta('10day')] # just look at a 10 day window

In [54]:
clean = clean.groupby(['card_id']).filter(lambda x: x['merchant_id'].count() > 5) # make sure the sequence len > 5

In [55]:
clean.groupby(['card_id'])['purchase_date'].count().sort_values(ascending=False)

card_id
C_ID_8d24dbcfdf    76
C_ID_6cffeb146a    63
C_ID_94cf87a008    61
C_ID_bf52d3770a    60
C_ID_f099a38468    58
                   ..
C_ID_9859b31349     6
C_ID_98427d2aec     6
C_ID_9841775757     6
C_ID_98345c3904     6
C_ID_87eb22726e     6
Name: purchase_date, Length: 20700, dtype: int64

In [56]:
stats = clean.groupby(['card_id'])['purchase_date'].count().describe()
stats

count    20700.000000
mean        10.687826
std          5.463045
min          6.000000
25%          7.000000
50%          9.000000
75%         13.000000
max         76.000000
Name: purchase_date, dtype: float64

In [57]:
print("Total Actions: {}".format(len(clean)))
print("Total Users: {}".format(len(clean['card_id'].unique())))
print("Total Items: {}".format(len(clean['merchant_id'].unique())))
print("Avg length: {}".format(stats['mean']))
print("Max length: {}".format(stats['max']))

Total Actions: 221238
Total Users: 20700
Total Items: 60293
Avg length: 10.687826086956521
Max length: 76.0


## Notes
It can be seen that mean number of purchases per card is 89 with a standard deviation of 105. The minimum is 2 and maximum is 5582. Given that the CoSeRec has a maximum sequence length cutoff of 50 we may have to cut down on the sequence size for each user.

Next steps will be to order the sequences for each card by date and build the dataset with card_id as the first column which will just be row_id and then the sequence of merchant_id pairs per that card id. 

In [58]:
label_encoder = preprocessing.LabelEncoder()  # convert the merchant and card ids to integers
clean['merchant_id']= label_encoder.fit_transform(clean['merchant_id'])
clean['card_id']= label_encoder.fit_transform(clean['card_id'])

In [59]:
clean.head()

,card_id,merchant_id,purchase_date
18512762,19345,5746,2017-01-01 00:00:59
14942234,10449,19780,2017-01-01 00:01:41
20004812,2287,10622,2017-01-01 00:02:12
12436720,19735,154,2017-01-01 00:02:25
3719958,19130,25663,2017-01-01 00:02:25


In [60]:
groups = clean.groupby(['card_id']).merchant_id.apply(list)

In [61]:
groups # it is in order because we ordered clean by date previously

card_id
0            [57322, 9294, 35208, 154, 35208, 4374, 35208]
1               [20234, 51048, 43358, 30881, 24023, 24023]
2        [29126, 46092, 34156, 31062, 51246, 51246, 51246]
3               [30746, 30746, 53500, 30746, 36943, 30746]
4        [33297, 33297, 32478, 17152, 34764, 32478, 324...
                               ...                        
20695    [7178, 54521, 37522, 55212, 21913, 58328, 2191...
20696    [59259, 39268, 20775, 40017, 43739, 59259, 247...
20697    [34521, 34521, 34521, 34521, 52839, 49453, 52931]
20698          [482, 482, 25322, 482, 52471, 25322, 47358]
20699    [52934, 54257, 33668, 33, 33668, 22484, 154, 2...
Name: merchant_id, Length: 20700, dtype: object

In [62]:
file = open('data/card_purchases.txt', 'w')
writer = csv.writer(file)
max_seq_len = 50
for k in range(len(groups)):
    d = groups[k][:max_seq_len] # cut the sequence length
    d.insert(0, k) # this is not ideal because it's not immutable and it messes with groups
    writer.writerow(d)
file.close()